In [11]:
# Diagnostic reporting functions for Bayesian Analysis:
def report_posterior_stats(trace):
    print("Posterior Statistics:\n")
    summary = az.summary(trace, var_names=['mu', 'sigma'])
    print("Posterior Summary:\n", summary) 

def report_histogram_stats(parameter_samples, random_distribution='normal', interested='Mu'):
    print("Histogram Statistics:\n")
    random_dist = {'normal': np.random.normal(0, 0.01, 1000),
                   'gamma': np.random.gamma(2, 0.01, 1000),}
    
    # For Prior Param (from the 1000 samples used in histogram)
    prior_param_samples = random_dist.get(random_distribution, np.random.normal(0, 0.01, 1000))
    prior_param_hist, prior_param_bins = np.histogram(parameter_samples, bins=30)
    prior_param_mode_bin = prior_param_bins[np.argmax(prior_param_hist)]
    print(f"Prior {interested} - Min:", np.min(prior_param_samples), "Max:", np.max(prior_param_samples))
    print(f"Prior {interested} - Mode (approx bin center):", prior_param_mode_bin, "Count at mode:", np.max(prior_param_hist))

    # For Prior Param
    post_param_hist, post_param_bins = np.histogram(parameter_samples, bins=30)
    post_param_mode_bin = post_param_bins[np.argmax(post_param_hist)]
    print("Posterior {interested} - Min:", np.min(parameter_samples), "Max:", np.max(parameter_samples))
    print("Posterior {interested} - Mode (approx bin center):", post_param_mode_bin, "Count at mode:", np.max(post_param_hist))

def report_MCMCtrace(mu_samples):
    print("MCMC Trace Statistics:\n")
    print("Mu Trace - Min:", np.min(mu_samples), "Max:", np.max(mu_samples), "Mean:", np.mean(mu_samples))
    print("Mu Trace - Autocorrelation at lag 1:", pd.Series(mu_samples).autocorr(lag=1))  # Simple lag-1 autocorr; should be low for good mixing

def report_predictive_stats(pred_mean, pred_ci):
    print("Predictive Returns Statistics:\n")
    # Overall stats
    print("Predictive Returns - Overall Mean:", np.mean(pred_mean))
    print("Predictive Returns - Overall 95% CI Lower Mean:", np.mean(pred_ci[0]), "Upper Mean:", np.mean(pred_ci[1]))

    # Specific days for spot-check (e.g., day 1 and day 30; 0-indexed)
    print("Predictive Returns Day 1 - Mean:", pred_mean[0], "95% CI:", pred_ci[:, 0])
    print("Predictive Returns Day 30 - Mean:", pred_mean[29], "95% CI:", pred_ci[:, 29])

def report_posterior_density(mu_kde_vals, sigma_kde_vals, mu_samples, sigma_samples):
    print("Posterior Density Statistics:\n")
    print("Posterior Mu KDE - Max Density Value:", np.max(mu_kde_vals))
    print("Posterior Mu KDE - Mode (approx):", x_mu[np.argmax(mu_kde_vals)])
    print("Posterior Mu KDE - Range:", np.min(mu_samples), "to", np.max(mu_samples))
    print("Posterior Sigma KDE - Max Density Value:", np.max(sigma_kde_vals))
    print("Posterior Sigma KDE - Mode (approx):", x_sigma[np.argmax(sigma_kde_vals)])
    print("Posterior Sigma KDE - Range:", np.min(sigma_samples), "to", np.max(sigma_samples))

def report_autocorr(mu_samples, sigma_samples):
    from statsmodels.tsa.stattools import acf
    print("Autocorrelation Analysis:\n")
    mu_acf = acf(mu_samples, nlags=20, fft=False)
    sigma_acf = acf(sigma_samples, nlags=20, fft=False)
    print("Mu Autocorrelation (Lags 0-20):", mu_acf)
    print("Sigma Autocorrelation (Lags 0-20):", sigma_acf)
    # Check if autocorrelations drop below threshold (e.g., |0.1|)
    print("Mu Autocorrelation - Lags where |ACF| < 0.1:", np.where(np.abs(mu_acf) < 0.1)[0])
    print("Sigma Autocorrelation - Lags where |ACF| < 0.1:", np.where(np.abs(sigma_acf) < 0.1)[0])

def report_posterior_predictive(observed_returns, simulated_returns):
    from scipy.stats import ks_2samp
    print("Posterior Predictive Check (PPC) Statistics:\n")
    print("PPC - Observed Returns Mean:", np.mean(observed_returns))
    print("PPC - Observed Returns Std:", np.std(observed_returns))
    print("PPC - Simulated Returns Mean:", np.mean(simulated_returns))
    print("PPC - Simulated Returns Std:", np.std(simulated_returns))
    # Kolmogorov-Smirnov test for distributional similarity
    ks_stat, ks_pval = ks_2samp(observed_returns, simulated_returns)
    print("PPC - KS Test Statistic:", ks_stat, "P-value:", ks_pval)

In [12]:
# All-in-one plotting function: 
def save_fig_html(fig, filename:str, title:str, extension='html',path=None):
    import os 
    if path and os.path.isdir(path):
        filename = f"{path}/{filename}"
    fig.update_layout(title=title, showlegend=True)
    fig.write_html(f'{filename}.{extension}')  # Export for GitHub Pages


Scenario #01

In [16]:
import yfinance as yf
import pandas as pd
import numpy as np

# 1. Load Data
df = yf.download('AAPL', start='2020-06-01', end='2025-06-30')  # New timeframe: June 2020–June 2025


/tmp/ipykernel_686548/3989990771.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [35]:
df.Close.shift(1) 

Ticker,AAPL
Date,
2020-06-01,NaN
2020-06-02,78.126457
2020-06-03,78.488136
2020-06-04,78.920197
2020-06-05,78.240532
...,...
2025-06-23,200.772141
2025-06-24,201.271576
2025-06-25,200.072937


In [36]:
df['LogReturn'] = np.log(df['Close'].values / df['Close'].shift(1).values)
returns = df['LogReturn'].dropna().values  # ~1250 daily returns

df


Price,Close,High,Low,Open,Volume,LogReturn
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,
Date,,,,,,
2020-06-01,78.126457,78.247828,77.000132,77.131214,80791200,NaN
2020-06-02,78.488136,78.512412,77.417644,77.859436,87642800,0.004619
2020-06-03,78.920197,79.182362,78.235663,78.808537,104491200,0.005490
2020-06-04,78.240532,79.041577,77.866708,78.743009,87560400,-0.008649
2020-06-05,80.468887,80.529573,78.461416,78.490544,137250400,0.028083
...,...,...,...,...,...,...
2025-06-23,201.271576,202.070672,198.734462,201.401433,55814300,0.002484
2025-06-24,200.072937,203.209377,199.973044,202.360334,54064000,-0.005973


In [37]:
import yfinance as yf
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# 1. Load Data
df = yf.download('AAPL', start='2020-09-21', end='2025-09-21')
trading_freq = 1 # Daily trading: 1, Weekly: 5, Monthly: 21
df['LogReturn'] = np.log(df['Close'].values / df['Close'].shift(trading_freq).values)
returns = df['LogReturn'].dropna().values  # ~1,260 daily returns

# 2. Bayesian Volatility Model
with pm.Model() as model:
    # Priors: Normal-Inverse-Gamma for mean (mu) and variance (sigma2)
    mu = pm.Normal('mu', mu=0, sigma=0.01)
    sigma = pm.InverseGamma('sigma', alpha=2, beta=0.1)
    # Likelihood
    returns_obs = pm.Normal('returns_obs', mu=mu, sigma=sigma, observed=returns)
    # Predictive variable for future returns
    pred_returns = pm.Normal('pred_returns', mu=mu, sigma=sigma, shape=30)  # 30-day forecast
    # Sample posterior
    trace = pm.sample(1000, tune=1000, return_inferencedata=True)
    report_posterior_stats(trace) 

# 3. Posterior Predictive Sampling
with model:
    pred_trace = pm.sample_posterior_predictive(trace, var_names=['pred_returns'])

# 4. Extract Posterior and Predictive Samples
posterior = az.extract(trace)
mu_samples = posterior['mu'].values  # Shape: (4000,) after flattening chains

sigma_samples = posterior['sigma'].values  # Shape: (4000,)
pred_samples = pred_trace.posterior_predictive['pred_returns'].values  # Shape: (chains, draws, 30)

# Save trace for reproducibility
az.to_netcdf(trace, 'trace.nc')
az.to_netcdf(pred_trace, 'pred_trace.nc')

# 5. Interactive Visualization
#subplot_titles = ['Prior vs Posterior: Mu', 
#                  'Prior vs Posterior: Sigma', 
#                  'MCMC Trace: Mu', 
#                  'Predictive Returns (30 Days)', 
#                  'Posterior Density Plot',
#                  'Autocorrelations']
#fig = make_subplots(rows=1, cols=1, subplot_titles=subplot_titles)

# Prior vs Posterior: Mu
row, col= 1, 1
fig = make_subplots(rows=row, cols=col)
fig.add_trace(go.Histogram(x=np.random.normal(0, 0.01, 1000), name='Prior Mu', opacity=0.5), row=row, col=col)
fig.add_trace(go.Histogram(x=mu_samples, name='Posterior Mu', opacity=0.5), row=row, col=col)
## Add Posterior Predictive returns histogram
fig.add_trace(go.Histogram(x=returns, name='Observed Returns', opacity=0.5, nbinsx=50), row=row, col=col)  # Adjust row/col as needed
simulated_returns = pred_trace.posterior_predictive['pred_returns'].values.flatten()[:1000]  # Subsample for clarity
fig.add_trace(go.Histogram(x=simulated_returns, name='Simulated Returns', opacity=0.5, nbinsx=50), row=row, col=col)
save_fig_html(fig, 's1_prior_post_mu', 'Prior vs Posterior: Average(Mu) Daily Return')
report_histogram_stats(mu_samples, random_distribution='normal', interested='Mu')

# Prior vs Posterior: Sigma
fig = make_subplots(rows=row, cols=col)
fig.add_trace(go.Histogram(x=np.random.gamma(2, 0.1, 1000), name='Prior Sigma', opacity=0.5), row=row, col=col)
fig.add_trace(go.Histogram(x=sigma_samples, name='Posterior Sigma', opacity=0.5), row=row, col=col)
save_fig_html(fig, 's1_prior_post_sigma', 'Prior vs Posterior: Volatility(Sigma) of Daily Return', path='../plots')
report_histogram_stats(sigma_samples, random_distribution='gamma', interested='Sigma')

# MCMC Trace: Mu
fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Scatter(x=np.arange(len(mu_samples)), y=mu_samples, mode='lines', name='Mu Trace'), row=row, col=col)
save_fig_html(fig, 's1_mcmc_trace_mu', 'MCMC Trace: Mu', path='../plots') 
report_MCMCtrace(mu_samples)

# Predictive Returns: Mean and 95% CI
fig = make_subplots(rows=1, cols=1) 
pred_mean = pred_samples.mean(axis=(0, 1))  # Mean over chains and draws
pred_ci = np.percentile(pred_samples, [2.5, 97.5], axis=(0, 1))  # 95% credible interval
report_predictive_stats(pred_mean, pred_ci)

fig.add_trace(go.Scatter(x=np.arange(30), y=pred_mean, mode='lines', name='Mean Pred Returns'), row=row, col=col)
fig.add_trace(go.Scatter(x=np.arange(30), y=pred_ci[0], mode='lines', name='95% CI Lower', line=dict(dash='dash')), row=row, col=col)
fig.add_trace(go.Scatter(x=np.arange(30), y=pred_ci[1], mode='lines', name='95% CI Upper', line=dict(dash='dash')), row=row, col=col)
save_fig_html(fig, 's1_predictive_returns', 'Predictive Returns (30 Days)', path='../plots')

# Autocorrelations
from statsmodels.tsa.stattools import acf
# Autocorrelation
mu_acf = acf(mu_samples, nlags=20, fft=False)
sigma_acf = acf(sigma_samples, nlags=20, fft=False)
fig = make_subplots(rows=1, cols=1) 
fig.add_trace(go.Bar(x=np.arange(21), y=mu_acf, name='Mu Autocorrelation'), row=row, col=col)  # Adjust row/col
fig.add_trace(go.Bar(x=np.arange(21), y=sigma_acf, name='Sigma Autocorrelation'), row=row, col=col)
#fig.update_xaxes(title_text="Lag", row=1, col=2)
#fig.update_yaxes(title_text="Autocorrelation", row=1, col=2)
save_fig_html(fig, 's1_autocorrelations', 'Autocorrelations (Mu and Sigma)', path='../plots')
report_autocorr(mu_samples, sigma_samples) 


# Posterior Density Plot: KDE for Mu and Sigma
import scipy.stats as stats
fig = make_subplots(rows=1, cols=1) 
mu_kde = stats.gaussian_kde(mu_samples)
sigma_kde = stats.gaussian_kde(sigma_samples)
x_mu = np.linspace(np.min(mu_samples), np.max(mu_samples), 100)
x_sigma = np.linspace(np.min(sigma_samples), np.max(sigma_samples), 100)
mu_kde_vals = mu_kde(x_mu) # Evaluate KDE on grid 
sigma_kde_vals = sigma_kde(x_sigma) # Evaluate KDE on grid 

fig.add_trace(go.Scatter(x=x_mu, y=mu_kde(x_mu), mode='lines', name='Posterior Mu KDE'), row=row, col=col)  # Adjust row/col
fig.add_trace(go.Scatter(x=x_sigma, y=sigma_kde(x_sigma), mode='lines', name='Posterior Sigma KDE'), row=row, col=col)
# Add HDI lines (from az.summary: mu [-0.000, 0.002], sigma [0.017, 0.019])
fig.add_vline(x=-0.000, line_dash="dash", line_color="blue", row=row, col=col)
fig.add_vline(x=0.002, line_dash="dash", line_color="blue", row=row, col=col)
fig.add_vline(x=0.017, line_dash="dash", line_color="orange", row=row, col=col)
fig.add_vline(x=0.019, line_dash="dash", line_color="orange", row=row, col=col)
#fig.update_xaxes(title_text="Parameter Value", row=2, col=2)
#fig.update_yaxes(title_text="Density", row=2, col=2)
save_fig_html(fig, 's1_post_density', 'Posterior Density (Mu and Sigma)', path='../plots')
report_posterior_density(mu_kde_vals, sigma_kde_vals, mu_samples, sigma_samples)


/tmp/ipykernel_686548/1244083656.py:10: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu, sigma, pred_returns]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 3 seconds.
Sampling: [pred_returns]


Output()

Posterior Statistics:

Posterior Summary:
         mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
mu     0.001  0.001  -0.000    0.002        0.0      0.0   10519.0    3245.0   
sigma  0.018  0.000   0.017    0.019        0.0      0.0    9055.0    2557.0   

       r_hat  
mu       1.0  
sigma    1.0  


Histogram Statistics:

Prior Mu - Min: -0.026156089614560993 Max: 0.03174871547173786
Prior Mu - Mode (approx bin center): 0.0007245348567461118 Count at mode: 410
Posterior {interested} - Min: -0.0012386010469329954 Max: 0.002442278772465331
Posterior {interested} - Mode (approx bin center): 0.0007245348567461118 Count at mode: 410
Histogram Statistics:

Prior Sigma - Min: 0.0006177196842074164 Max: 0.09871193953086606
Prior Sigma - Mode (approx bin center): 0.01806057473078925 Count at mode: 423
Posterior {interested} - Min: 0.01664015871321798 Max: 0.019480990748360523
Posterior {interested} - Mode (approx bin center): 0.01806057473078925 Count at mode: 423
MCMC Trace Statistics:

Mu Trace - Min: -0.0012386010469329954 Max: 0.002442278772465331 Mean: 0.0006546970527129747
Mu Trace - Autocorrelation at lag 1: -0.42950909085500644
Predictive Returns Statistics:

Predictive Returns - Overall Mean: 0.0006700159162032692
Predictive Returns - Overall 95% CI Lower Mean: -0.0346955402787585

---
Scenario #02

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# 1. Load and Preprocess Data (Updated URL)
from ucimlrepo import fetch_ucirepo
online_retail = fetch_ucirepo(id=352)  # UCI ID for Online Retail
df = online_retail.data.original  # Raw dataframe
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Compute time-to-next-purchase
df = df.sort_values(['CustomerID', 'InvoiceDate'])
df['NextPurchase'] = df.groupby('CustomerID')['InvoiceDate'].shift(-1)
df['TimeToNext'] = (df['NextPurchase'] - df['InvoiceDate']).dt.days
df['Censored'] = df['TimeToNext'].isna().astype(int)  # 1 if no next purchase (censored)
df['TimeToNext'] = df['TimeToNext'].fillna(365)  # Censor at 1 year
df['TimeToNext'] = df['TimeToNext'].clip(lower=0.1)  # Ensure times > 0 (add 0.1 for same-day purchases)

# Aggregate by customer, select top countries
df['Country'] = df['Country'].replace(['EIRE', 'Channel Islands'], 'Other')
top_countries = df['Country'].value_counts().head(5).index
df = df[df['Country'].isin(top_countries)]
customer_data = df.groupby(['CustomerID', 'Country']).agg({
    'TimeToNext': 'min',
    'Censored': 'min'
}).reset_index()

# Encode countries and validate shapes
customer_data = customer_data.reset_index(drop=True)
country_idx = pd.Categorical(customer_data['Country']).codes
times = customer_data['TimeToNext'].values
censored = customer_data['Censored'].values
n_customers = len(customer_data)
assert len(country_idx) == len(times) == len(censored) == n_customers, "Shape mismatch in data arrays"
assert all(times > 0), f"Invalid times: {times[times <= 0]}"  # Ensure all times positive

# 2. Bayesian Weibull Survival Model
with pm.Model() as survival_model:
    # Hyperpriors for country-level parameters
    mu_alpha = pm.HalfNormal('mu_alpha', sigma=2)  # Positive prior
    sigma_alpha = pm.HalfNormal('sigma_alpha', sigma=1)
    mu_beta = pm.HalfNormal('mu_beta', sigma=2)    # Positive prior
    sigma_beta = pm.HalfNormal('sigma_beta', sigma=1)
    
    # Country-specific parameters (positive)
    n_countries = len(np.unique(country_idx))
    alpha = pm.HalfNormal('alpha', sigma=sigma_alpha, shape=n_countries)
    beta = pm.HalfNormal('beta', sigma=sigma_beta, shape=n_countries)
    
    # Map parameters to customers
    alpha_i = alpha[country_idx]
    beta_i = beta[country_idx]
    
    # Weibull likelihood for uncensored data
    pm.Weibull('t_uncensored', alpha=alpha_i[censored == 0], beta=beta_i[censored == 0], 
               observed=times[censored == 0])
    
    # Log-survival for censored data
    censored_logsurv = -((times[censored == 1] / beta_i[censored == 1]) ** alpha_i[censored == 1])
    pm.Potential('censored_logsurv', censored_logsurv.sum())
    
    # Predictive survival times
    pred_times = pm.Weibull('pred_times', alpha=alpha_i, beta=beta_i, shape=n_customers)
    
    # Sample posterior
    trace = pm.sample(1000, tune=1000, return_inferencedata=True, target_accept=0.9)

# 3. Posterior Predictive Sampling
with survival_model:
    pred_trace = pm.sample_posterior_predictive(trace, var_names=['pred_times'])

# 4. Extract Posterior and Predictive Samples
posterior = az.extract(trace)
alpha_samples = posterior['alpha'].values  # Shape: (chains*draws, n_countries)
beta_samples = posterior['beta'].values    # Shape: (chains*draws, n_countries)
pred_times_samples = pred_trace.posterior_predictive['pred_times'].values  # Shape: (chains, draws, n_customers)

# Save traces
az.to_netcdf(trace, 'survival_trace.nc')
az.to_netcdf(pred_trace, 'survival_pred_trace.nc')

# 5. Interactive Visualization
fig = make_subplots(rows=1, cols=2, subplot_titles=('Posterior Alpha by Country', 'Survival Curves by Country'))

# Posterior Alpha
countries = pd.Categorical(customer_data['Country']).categories
colors = ['blue', 'red', 'green', 'orange', 'purple']
for i, country in enumerate(countries):
    fig.add_trace(go.Histogram(x=alpha_samples[:, i], name=f'Alpha: {country}', 
                               marker_color=colors[i], opacity=0.6), rows=row, cols=col)

# Survival Curves with Slider
t = np.linspace(0.1, 365, 100)  # Start at 0.1 to avoid log(0)
traces = []
for i, country in enumerate(countries):
    alpha_mean = alpha_samples[:, i].mean()
    beta_mean = beta_samples[:, i].mean()
    survival = np.exp(-((t / beta_mean) ** alpha_mean))
    trace = go.Scatter(x=t, y=survival, mode='lines', name=f'Survival: {country}',
                       line=dict(color=colors[i]), visible=(i == 0))
    traces.append(trace)
    fig.add_trace(trace, row=1, col=2)

# Slider for country selection
steps = [
    {'method': 'restyle', 'label': country, 'args': [{'visible': [j == i for j in range(len(countries))]}]}
    for i, country in enumerate(countries)
]
fig.update_layout(
    title='Bayesian Survival Analysis for Customer Lifetime Value',
    xaxis2_title='Days to Next Purchase', yaxis2_title='Survival Probability',
    sliders=[{'steps': steps, 'active': 0, 'currentvalue': {'prefix': 'Country: '}}],
    showlegend=True
)
fig.write_html('survival_viz.html')

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu_alpha, sigma_alpha, mu_beta, sigma_beta, alpha, beta, pred_times]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 223 seconds.
/tmp/ipykernel_29394/3570969171.py:73: UserWarning: The effect of Potentials on other parameters is ignored during posterior predictive sampling. This is likely to lead to invalid or biased predictive samples.
  pred_trace = pm.sample_posterior_predictive(trace, var_names=['pred_times'])
Sampling: [pred_times]


Output()

---
Scenario #3

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler

# 1. Load and Preprocess Data
url = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"
df = pd.read_csv(url)
# Handle missing TotalCharges (replace empty strings with NaN, then impute)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())
# Select features: tenure, MonthlyCharges, Contract (categorical)
df['Contract'] = df['Contract'].map({'Month-to-month': 0, 'One year': 1, 'Two year': 2})
df = df[['tenure', 'MonthlyCharges', 'Contract', 'Churn']].dropna()
# Encode Churn (Yes=1, No=0)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
# Standardize numerical features
scaler = StandardScaler()
df[['tenure', 'MonthlyCharges']] = scaler.fit_transform(df[['tenure', 'MonthlyCharges']])
# Prepare data arrays
X = df[['tenure', 'MonthlyCharges', 'Contract']].values
y = df['Churn'].values
n_samples, n_features = X.shape
assert len(y) == n_samples, "Mismatch in X and y shapes"

# 2. Bayesian Logistic Regression Model
with pm.Model() as churn_model:
    # Priors for coefficients
    beta = pm.Normal('beta', mu=0, sigma=2, shape=n_features)
    intercept = pm.Normal('intercept', mu=0, sigma=2)
    # Linear combination
    logits = pm.math.dot(X, beta) + intercept
    # Likelihood
    pm.Bernoulli('churn', logit_p=logits, observed=y)
    # Posterior predictive for probabilities
    churn_prob = pm.Bernoulli('churn_prob', logit_p=logits, shape=n_samples)
    # Sample posterior
    trace = pm.sample(1000, tune=1000, return_inferencedata=True, target_accept=0.9)

# 3. Posterior Predictive Sampling
with churn_model:
    pred_trace = pm.sample_posterior_predictive(trace, var_names=['churn_prob'])

# 4. Extract Posterior and Predictive Samples
posterior = az.extract(trace)
beta_samples = posterior['beta'].values  # Shape: (chains*draws, n_features)
pred_probs = pred_trace.posterior_predictive['churn_prob'].mean(axis=(0, 1))  # Mean probability per customer

# Save traces
az.to_netcdf(trace, 'churn_trace.nc')
az.to_netcdf(pred_trace, 'churn_pred_trace.nc')

# 5. Interactive Visualization (ROC Curve with Threshold Slider)
# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y, pred_probs)
roc_auc = auc(fpr, tpr)

fig = make_subplots(rows=1, cols=2, subplot_titles=('Posterior Beta Coefficients', 'ROC Curve with Threshold Slider'))

# Posterior Beta Coefficients
feature_names = ['tenure', 'MonthlyCharges', 'Contract']
colors = ['blue', 'red', 'green']
for i, name in enumerate(feature_names):
    fig.add_trace(go.Histogram(x=beta_samples[:, i], name=f'Beta: {name}', 
                               marker_color=colors[i], opacity=0.6), rows=row, cols=col)

# ROC Curve
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC Curve (AUC = {roc_auc:.2f})',
                         line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Guess',
                         line=dict(color='black', dash='dash')), row=1, col=2)

# Add classification point for threshold
thresholds_subset = np.linspace(0, 1, 11)  # 0.0 to 1.0 in steps of 0.1
traces = []
for thresh in thresholds_subset:
    idx = np.argmin(np.abs(thresholds - thresh))
    trace = go.Scatter(x=[fpr[idx]], y=[tpr[idx]], mode='markers', name=f'Threshold: {thresh:.2f}',
                       marker=dict(size=10, color='red'), visible=(thresh == 0.5))
    traces.append(trace)
    fig.add_trace(trace, row=1, col=2)

# Slider for classification threshold
steps = [
    {'method': 'restyle', 'label': f'{thresh:.2f}', 
     'args': [{'visible': [True, True] + [j == i for j in range(len(thresholds_subset))]}]}
    for i, thresh in enumerate(thresholds_subset)
]
fig.update_layout(
    title='Bayesian Churn Prediction for Telco Customers',
    xaxis2_title='False Positive Rate', yaxis2_title='True Positive Rate',
    sliders=[{'steps': steps, 'active': 5, 'currentvalue': {'prefix': 'Threshold: '}}],
    showlegend=True
)
fig.write_html('churn_viz.html')


/root/anaconda3/lib/python3.12/site-packages/pytensor/link/c/cmodule.py:2968: UserWarning: PyTensor could not link to a BLAS installation. Operations that might benefit from BLAS will be severely degraded.
This usually happens when PyTensor is installed via pip. We recommend it be installed via conda/mamba/pixi instead.
Alternatively, you can use an experimental backend such as Numba or JAX that perform their own BLAS optimizations, by setting `pytensor.config.mode == 'NUMBA'` or passing `mode='NUMBA'` when compiling a PyTensor function.
For more options and details see https://pytensor.readthedocs.io/en/latest/troubleshooting.html#how-do-i-configure-test-my-blas-library
  warnings.warn(
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>NUTS: [beta, intercept]
>BinaryGibbsMetropolis: [churn_prob]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 2522 seconds.
/root/anaconda3/lib/python3.12/site-packages/arviz/stats/diagnostics.py:596: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
Sampling: [churn_prob]


Output()

---
Scenario #4


In [ ]:
# Since the above cell take too much time to run, let's run simpler version:
# - size down: n_users, n_songs = 500, 500 
# - Instead of pm.Poisson likelihood, use pm.Normal likelihood on log transformed counts.
# - Fewer latent dimensions: n_latent = 5 
# - Reduced tuning steps: tune=500 
# - Sparse matrix: csr_matrix for preprocessing 
# - Numerical stability: 1e-8 to norms and clipped sim_scores to [-1, 1]

import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.manifold import TSNE
import requests
import zipfile
import io
import pytensor.tensor as pt
from scipy.sparse import csr_matrix

# 1. Load and Preprocess Data (Download and Unzip)
url = "http://labrosa.ee.columbia.edu/~dpwe/tmp/train_triplets.txt.zip"
response = requests.get(url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open('train_triplets.txt') as f:
        df = pd.read_csv(f, sep='\t', names=['user', 'song', 'count'])

# Subsample for efficiency
n_users, n_songs = 500, 500  # Reduced from 1000
top_users = df['user'].value_counts().head(n_users).index
top_songs = df['song'].value_counts().head(n_songs).index
df = df[df['user'].isin(top_users) & df['song'].isin(top_songs)]
# Create sparse user-song matrix
play_counts = df.pivot_table(index='user', columns='song', values='count', fill_value=0)
play_counts = csr_matrix(play_counts.values)  # Sparse matrix
user_ids = pd.Categorical(df['user'].unique()).categories[:n_users]
song_ids = pd.Categorical(df['song'].unique()).categories[:n_songs]
n_samples = play_counts.shape[0]
assert play_counts.shape == (n_users, n_songs), "Play count matrix shape mismatch"

# Transform counts to stabilize variance (log1p for sparse data)
play_counts_dense = np.log1p(play_counts.toarray())  # Log-transform for Normal likelihood

# 2. Bayesian ABC Model
n_latent = 5  # Reduced latent dimensions for speed
with pm.Model() as abc_model:
    # Priors for latent user preferences and song features
    user_prefs = pm.Normal('user_prefs', mu=0, sigma=1, shape=(n_users, n_latent))
    song_features = pm.Normal('song_features', mu=0, sigma=1, shape=(n_songs, n_latent))
    
    # Compute cosine similarity (vectorized)
    dot_product = pm.math.sum(user_prefs[:, None, :] * song_features[None, :, :], axis=2)
    user_norm = pm.math.sqrt(pm.math.sum(user_prefs**2, axis=1))
    song_norm = pm.math.sqrt(pm.math.sum(song_features**2, axis=1))
    sim_scores = dot_product / (user_norm[:, None] * song_norm[None, :] + 1e-8)  # Avoid division by zero
    sim_scores = pm.math.clip(sim_scores, -1, 1)
    
    # Expected counts (Normal likelihood for log-transformed counts)
    mu = 3 * sim_scores  # Scale similarity to match log-count scale
    pm.Normal('observed_counts', mu=mu, sigma=1, observed=play_counts_dense)
    
    # Sample using ABC
    trace = pm.sample(1000, tune=500, return_inferencedata=True, target_accept=0.95)

# 3. Posterior Predictive Sampling
with abc_model:
    pred_trace = pm.sample_posterior_predictive(trace, var_names=['observed_counts'])

# 4. Extract Posterior and Predictive Samples
posterior = az.extract(trace)
user_prefs_samples = posterior['user_prefs'].values  # Shape: (chains*draws, n_users, n_latent)
song_features_samples = posterior['song_features'].values  # Shape: (chains*draws, n_songs, n_latent)
pred_counts = np.expm1(pred_trace.posterior_predictive['observed_counts'].mean(axis=(0, 1)))  # Inverse log1p

# Save traces
az.to_netcdf(trace, 'music_trace.nc')
az.to_netcdf(pred_trace, 'music_pred_trace.nc')

# 5. Interactive Visualization (t-SNE with ABC Animation)
tsne = TSNE(n_components=2, random_state=42)
user_prefs_mean = user_prefs_samples.mean(axis=0)  # Shape: (n_users, n_latent)
user_tsne = tsne.fit_transform(user_prefs_mean)  # Shape: (n_users, 2)

fig = make_subplots(rows=1, cols=2, subplot_titles=('User Preference Posterior', 't-SNE User Embeddings'))

# Posterior for one user’s latent dimension
fig.add_trace(go.Histogram(x=user_prefs_samples[:, 0, 0], name='User 0: Latent Dim 1', 
                           marker_color='blue', opacity=0.6), rows=row, cols=col)

# t-SNE Scatter
fig.add_trace(go.Scatter(x=user_tsne[:, 0], y=user_tsne[:, 1], mode='markers',
                         marker=dict(size=5, color='blue'), name='Users'), row=1, col=2)

# Animation for posterior draws (simplified to 5 frames)
n_frames = 5
draws = np.linspace(0, user_prefs_samples.shape[0] - 1, n_frames, dtype=int)
frames = []
for i, draw in enumerate(draws):
    tsne_frame = TSNE(n_components=2, random_state=42).fit_transform(user_prefs_samples[draw])
    frame = go.Frame(
        data=[go.Scatter(x=tsne_frame[:, 0], y=tsne_frame[:, 1], mode='markers',
                         marker=dict(size=5, color='blue'), name='Users')],
        name=f'Draw {i}'
    )
    frames.append(frame)

fig.update(frames=frames)
fig.update_layout(
    title='Bayesian ABC for Music Recommendations',
    xaxis2_title='t-SNE Dim 1', yaxis2_title='t-SNE Dim 2',
    updatemenus=[{
        'buttons': [
            {'method': 'animate', 'label': 'Play', 'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}]},
            {'method': 'animate', 'label': 'Pause', 'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}]}
        ],
        'direction': 'left', 'pad': {'r': 10, 't': 87}, 'showactive': True, 'type': 'buttons'
    }],
    showlegend=True
)
fig.write_html('music_viz.html')

Initializing NUTS using jitter+adapt_diag...
/root/anaconda3/lib/python3.12/site-packages/pytensor/link/c/cmodule.py:2968: UserWarning: PyTensor could not link to a BLAS installation. Operations that might benefit from BLAS will be severely degraded.
This usually happens when PyTensor is installed via pip. We recommend it be installed via conda/mamba/pixi instead.
Alternatively, you can use an experimental backend such as Numba or JAX that perform their own BLAS optimizations, by setting `pytensor.config.mode == 'NUMBA'` or passing `mode='NUMBA'` when compiling a PyTensor function.
For more options and details see https://pytensor.readthedocs.io/en/latest/troubleshooting.html#how-do-i-configure-test-my-blas-library
  warnings.warn(
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [user_prefs, song_features]


Output()

Sampling 4 chains for 500 tune and 1_000 draw iterations (2_000 + 4_000 draws total) took 18211 seconds.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
Chain 0 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain 1 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain 2 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain 3 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Sampling: [observed_counts]


Output()

ValueError: perplexity must be less than n_samples

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.manifold import TSNE
import requests
import zipfile
import io
from scipy.sparse import csr_matrix
import logging
import time
import warnings
import pytensor
import sys
import xarray as xr
import os 

# Disable rich console output in Jupyter to prevent recursion
try:
    from rich.console import Console
    Console()._live = None  # Disable rich's live display
except ImportError:
    pass

# Redirect warnings to avoid recursive output
warnings.filterwarnings('ignore', category=UserWarning, module='pytensor')

# Set PyTensor BLAS config to avoid KeyError
pytensor.config.blas__ldflags = ''

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger("pymc")

# 1. Load and Preprocess Data (Download and Unzip)
logger.info("Loading and unzipping dataset...")
start_time = time.time()
if os.path.exists('train_triplets.txt'):
    df = pd.read_csv('train_triplets.txt', sep='\t', names=['user', 'song', 'count'])
else:
    url = "http://labrosa.ee.columbia.edu/~dpwe/tmp/train_triplets.txt.zip"
    response = requests.get(url)
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        with z.open('train_triplets.txt') as f:
            df = pd.read_csv(f, sep='\t', names=['user', 'song', 'count'])
logger.info(f"Data loaded in {time.time() - start_time:.2f} seconds")

# Subsample for efficiency
n_users, n_songs = 50, 50  # Reduced to minimize computation
logger.info(f"Subsampling {n_users} users and {n_songs} songs...")
top_users = df['user'].value_counts().head(n_users + 50).index
top_songs = df['song'].value_counts().head(n_songs + 50).index
df = df[df['user'].isin(top_users) & df['song'].isin(top_songs)]

# Create user-song matrix and reindex
play_counts = df.pivot_table(index='user', columns='song', values='count', fill_value=0)
all_users = top_users[:n_users]
all_songs = top_songs[:n_songs]
play_counts = play_counts.reindex(index=all_users, columns=all_songs, fill_value=0)
play_counts = csr_matrix(play_counts.values)
user_ids = pd.Categorical(all_users).categories
song_ids = pd.Categorical(all_songs).categories
assert play_counts.shape == (n_users, n_songs), f"Play count matrix shape {play_counts.shape}, expected {(n_users, n_songs)}"
assert len(user_ids) == n_users, f"Got {len(user_ids)} users, expected {n_users}"
assert len(song_ids) == n_songs, f"Got {len(song_ids)} songs, expected {n_songs}"
logger.info(f"Play counts matrix shape: {play_counts.shape}")

# Normalize counts to [0, 1]
play_counts_dense = play_counts.toarray()
play_counts_norm = play_counts_dense / (play_counts_dense.max() + 1e-8)

# 2. Bayesian ABC Model (Matrix Factorization)
n_latent = 3
logger.info("Starting SMC sampling...")
with pm.Model() as abc_model:
    user_prefs = pm.Normal('user_prefs', mu=0, sigma=1, shape=(n_users, n_latent))
    song_features = pm.Normal('song_features', mu=0, sigma=1, shape=(n_songs, n_latent))
    mu = pm.math.sigmoid(pm.math.dot(user_prefs, song_features.T))
    pm.Normal('observed_counts', mu=mu, sigma=0.1, observed=play_counts_norm)
    start_time = time.time()
    trace = pm.sample_smc(500, cores=4, return_inferencedata=True, progressbar=True)
    logger.info(f"SMC sampling completed in {time.time() - start_time:.2f} seconds")

# 3. Posterior Predictive Sampling
logger.info("Starting posterior predictive sampling...")
with abc_model:
    pred_trace = pm.sample_posterior_predictive(trace, var_names=['observed_counts'])
logger.info(f"Posterior predictive sampling completed in {time.time() - start_time:.2f} seconds")

# 4. Extract Posterior and Predictive Samples
logger.info("Raw trace posterior structure:")
logger.info(f"{trace.posterior}")
posterior = az.extract(trace, combined=True)  # Combine chains and draws
user_prefs_samples = np.asarray(posterior['user_prefs'].values, dtype=np.float64)  # Shape: (draws × chains, n_users, n_latent)
song_features_samples = np.asarray(posterior['song_features'].values, dtype=np.float64)  # Shape: (draws × chains, n_songs, n_latent)

# Check and correct dimension order
expected_draws = 500 * 4  # 500 draws × 4 chains
if user_prefs_samples.shape != (expected_draws, n_users, n_latent):
    logger.info(f"Transposing user_prefs_samples from {user_prefs_samples.shape} to ({expected_draws}, {n_users}, {n_latent})")
    user_prefs_samples = user_prefs_samples.transpose(2, 0, 1)
if song_features_samples.shape != (expected_draws, n_songs, n_latent):
    logger.info(f"Transposing song_features_samples from {song_features_samples.shape} to ({expected_draws}, {n_songs}, {n_latent})")
    song_features_samples = song_features_samples.transpose(2, 0, 1)

logger.info(f"user_prefs_samples shape: {user_prefs_samples.shape}, expected: ({expected_draws}, {n_users}, {n_latent})")
logger.info(f"song_features_samples shape: {song_features_samples.shape}, expected: ({expected_draws}, {n_songs}, {n_latent})")
assert user_prefs_samples.shape == (expected_draws, n_users, n_latent), f"Unexpected shape for user_prefs_samples: {user_prefs_samples.shape}"
assert song_features_samples.shape == (expected_draws, n_songs, n_latent), f"Unexpected shape for song_features_samples: {song_features_samples.shape}"
pred_counts = pred_trace.posterior_predictive['observed_counts'].mean(axis=(0, 1)) * play_counts_dense.max()

# Save traces as simplified InferenceData
logger.info("Saving traces to NetCDF...")
try:
    # Create separate Datasets for user_prefs and song_features
    user_prefs_ds = xr.Dataset(
        {
            'user_prefs': (['draw', 'user', 'latent'], user_prefs_samples)
        },
        coords={
            'draw': np.arange(user_prefs_samples.shape[0]),
            'user': np.arange(n_users),
            'latent': np.arange(n_latent)
        }
    )
    song_features_ds = xr.Dataset(
        {
            'song_features': (['draw', 'song', 'latent'], song_features_samples)
        },
        coords={
            'draw': np.arange(song_features_samples.shape[0]),
            'song': np.arange(n_songs),
            'latent': np.arange(n_latent)
        }
    )
    # Merge Datasets
    posterior_ds = xr.merge([user_prefs_ds, song_features_ds])
    # Save posterior trace
    posterior_ds.to_netcdf('music_trace.nc')
    # Save posterior predictive trace
    az.to_netcdf(pred_trace, 'music_pred_trace.nc')
    logger.info("Traces saved successfully")
except Exception as e:
    logger.error(f"Failed to save traces: {str(e)}")
    raise

# Check convergence
logger.info("Convergence diagnostics:")
print(az.summary(trace, var_names=['user_prefs', 'song_features'], round_to=2))

# 5. Interactive Visualization (t-SNE with ABC Animation)
logger.info("Generating t-SNE visualization...")
tsne = TSNE(n_components=2, random_state=42)
user_prefs_mean = user_prefs_samples.mean(axis=0)
user_tsne = tsne.fit_transform(user_prefs_mean)

fig = make_subplots(rows=1, cols=2, subplot_titles=('User Preference Posterior', 't-SNE User Embeddings'))

fig.add_trace(go.Histogram(x=user_prefs_samples[:, 0, 0], name='User 0: Latent Dim 1', 
                           marker_color='blue', opacity=0.6), rows=row, cols=col)
fig.add_trace(go.Scatter(x=user_tsne[:, 0], y=user_tsne[:, 1], mode='markers',
                         marker=dict(size=5, color='blue'), name='Users'), row=1, col=2)

# Animation for posterior draws (3 frames)
n_frames = 3
draws = np.linspace(0, user_prefs_samples.shape[0] - 1, n_frames, dtype=int)
frames = []
for i, draw in enumerate(draws):
    tsne_frame = TSNE(n_components=2, random_state=42).fit_transform(user_prefs_samples[draw])
    frame = go.Frame(
        data=[go.Scatter(x=tsne_frame[:, 0], y=tsne_frame[:, 1], mode='markers',
                         marker=dict(size=5, color='blue'), name='Users')],
        name=f'Draw {i}'
    )
    frames.append(frame)

fig.update(frames=frames)
fig.update_layout(
    title='Bayesian ABC for Music Recommendations',
    xaxis2_title='t-SNE Dim 1', yaxis2_title='t-SNE Dim 2',
    updatemenus=[{
        'buttons': [
            {'method': 'animate', 'label': 'Play', 'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}]},
            {'method': 'animate', 'label': 'Pause', 'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}]}
        ],
        'direction': 'left', 'pad': {'r': 10, 't': 87}, 'showactive': True, 'type': 'buttons'
    }],
    showlegend=True
)
fig.write_html('music_viz.html')
logger.info("Visualization saved as music_viz.html")

Loading and unzipping dataset...
INFO:pymc:Loading and unzipping dataset...
Data loaded in 119.32 seconds
INFO:pymc:Data loaded in 119.32 seconds
Subsampling 50 users and 50 songs...
INFO:pymc:Subsampling 50 users and 50 songs...
Play counts matrix shape: (50, 50)
INFO:pymc:Play counts matrix shape: (50, 50)
Starting SMC sampling...
INFO:pymc:Starting SMC sampling...
Initializing SMC sampler...
INFO:pymc.smc.sampling:Initializing SMC sampler...
Sampling 4 chains in 4 jobs
INFO:pymc.smc.sampling:Sampling 4 chains in 4 jobs


Output()

The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
INFO:pymc.stats.convergence:The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
SMC sampling completed in 25.24 seconds
INFO:pymc:SMC sampling completed in 25.24 seconds
Starting posterior predictive sampling...
INFO:pymc:Starting posterior predictive sampling...
Sampling: [observed_counts]
INFO:pymc.sampling.forward:Sampling: [observed_

Output()

Posterior predictive sampling completed in 25.48 seconds
INFO:pymc:Posterior predictive sampling completed in 25.48 seconds
Raw trace posterior structure:
INFO:pymc:Raw trace posterior structure:
<xarray.Dataset>
Dimensions:              (chain: 4, draw: 500, user_prefs_dim_0: 50,
                          user_prefs_dim_1: 3, song_features_dim_0: 50,
                          song_features_dim_1: 3)
Coordinates:
  * chain                (chain) int64 0 1 2 3
  * draw                 (draw) int64 0 1 2 3 4 5 6 ... 494 495 496 497 498 499
  * user_prefs_dim_0     (user_prefs_dim_0) int64 0 1 2 3 4 5 ... 45 46 47 48 49
  * user_prefs_dim_1     (user_prefs_dim_1) int64 0 1 2
  * song_features_dim_0  (song_features_dim_0) int64 0 1 2 3 4 ... 46 47 48 49
  * song_features_dim_1  (song_features_dim_1) int64 0 1 2
Data variables:
    user_prefs           (chain, draw, user_prefs_dim_0, user_prefs_dim_1) float64 ...
    song_features        (chain, draw, song_features_dim_0, song_features_dim_

                      mean    sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  \
user_prefs[0, 0]      0.29  0.71   -0.36     1.45       0.35     0.17   
user_prefs[0, 1]     -0.91  1.24   -2.93     0.47       0.62     0.31   
user_prefs[0, 2]     -0.01  0.48   -0.69     0.72       0.24     0.12   
user_prefs[1, 0]     -0.82  0.72   -1.46     0.39       0.35     0.18   
user_prefs[1, 1]      0.62  1.37   -1.77     1.71       0.68     0.39   
...                    ...   ...     ...      ...        ...      ...   
song_features[48, 1] -0.53  0.88   -1.21     1.00       0.44     0.25   
song_features[48, 2]  1.04  0.75   -0.26     1.79       0.37     0.20   
song_features[49, 0]  0.19  0.12    0.06     0.43       0.06     0.03   
song_features[49, 1]  0.28  0.53   -0.46     1.08       0.26     0.13   
song_features[49, 2]  0.36  0.95   -1.24     1.31       0.47     0.25   

                      ess_bulk  ess_tail  r_hat  
user_prefs[0, 0]          4.79     31.91   2.61  
user_prefs[0, 1]       

Visualization saved as music_viz.html
INFO:pymc:Visualization saved as music_viz.html
